# Figure 1. Synaptic input and upcrossing event.

Copyright (C) 2023 Robert Gowers and Magnus Richardson

This program is free software; you can redistribute it and/or modify it under the terms of the GNU General Public
License as published by the Free Software Foundation; either version 3 of the License, or (at your option) any later version.

This program is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with this program; if not, see <https://www.gnu.org/licenses>.

---  

In [1]:
# Packages
using PyPlot
using DelimitedFiles
using Statistics
using Distributions
using Random

# Load functions and model parameters common to all figures
include("CommonCode.jl");


------------------------------------------
EL=-60.0 	Ee=0.0  	Ei=-80.0
tauL=40.0 	taue=3.0 	taui=10.0
lamL=224.0 	lame=19.0 	lami=64.0
		game=0.1 	gami=0.1

kappa=0.6 tau0=24.0
D=1250.0
Vth=-50.0 Vre=-60.0
L=2000.0
------------------------------------------



---  

## Functions for this figure only    

In [2]:
function DendrShotSingleSynapseSim(x,t,xs,ts,game)
    
    dt=t[2]-t[1]; nt=length(t); 
    dx=x[2]-x[1]; nx=length(x)
    sh=Int(round(nx/2;digits=0)) # this is the central point
    
    # Initialise
    U=zeros(nx,nt);  U[:,1].=EL
    He=zeros(nx,nt); He[:,1].=0
    Hi=zeros(nx,nt); Hi[:,1].=0
    
    # useful constants
    HL=1/tauL
    D=lamL^2/tauL
    
    kts=maximum(findall(t.<=ts))
    kxs=maximum(findall(x.<=xs))
    
    for k=1:nt-1
        
        ##########################################
        # Synaptic input shared by all
        ##########################################
        He[:,k+1]=He[:,k] .- (dt/taue)*He[:,k] 
        Hi[:,k+1]=Hi[:,k] .- (dt/taui)*Hi[:,k]  
        if k+1==kts
            He[kxs,k+1]=He[kxs,k+1]+game/(2*pi*a*c*dx)
        end
        
        ##########################################
        # Voltage dynamics
        ##########################################
        HeU=He[:,k].*(Ee .-U[:,k])
        HiU=Hi[:,k].*(Ei .-U[:,k])
        HLU=(EL .-U[:,k])/tauL
        Dd2Udx2=D*diff(diff([U[1,k];U[:,k];U[end,k]]))/dx^2
        U[:,k+1]=U[:,k] .+ dt*( HLU .+ HeU .+ HiU .+ Dd2Udx2 )
        
    end
    
    return U,He,Hi

end;

In [3]:
# Single run for the point-neuron in the Gaussian approximation
function PointShotSingleSynapseSim(t,ts,game)

    dt=t[2]-t[1]
    nt=length(t); 
    
    # initialise
    U=zeros(nt);   U[1]=EL
    He=zeros(nt);  He[1]=0
    Hi=zeros(nt);  Hi[1]=0
    
    # the synaptic spike is here
    ks=maximum(findall(t.<=ts))
    
    for k=1:nt-1
        
        ##########################################
        # synaptic dynamics are used by all cases
        ##########################################
        He[k+1]=He[k] - (dt/taue)*He[k]
        Hi[k+1]=Hi[k] - (dt/taui)*Hi[k]
        if k==ks
            He[k+1]=He[k+1]+game/C
        end
        
        ##########################################
        # upcrossing quantities with cross terms
        ##########################################
        HeU=He[k].*(Ee -U[k])
        HiU=Hi[k].*(Ei -U[k])
        HLU=(EL-U[k])/tauL
        U[k+1]=U[k] + dt*(HLU + HeU + HiU)
        
    end
    
    return U,He,Hi

end;

In [4]:
# My analytical functions

function PointShotSingleSynapseTheory(t,game)
    dt=t[2]-t[1]
    U=EL .+ (taue*tauL*(Ee-EL)*game/((tauL-taue)*C))*( exp.(-t/tauL) .- exp.(-t./taue) )
    return U
end;

In [5]:
function DendShotSingleSynapseTheory(t,xs,game)
    
    dt=t[2]-t[1]
    betae=(Ee-EL)*tauL*game/(2*pi*a*c*lamL)
    tt=t .- dt/2
    tt=t
    Q=exp.(-tt/tauL .+ tt/taue .- ((xs/lamL)^2)./(4*tt/tauL))./sqrt.(4*pi*t/tauL)
    
    return EL .+ betae*(dt/tauL)*cumsum(Q).*exp.(-t/taue)

end;

In [6]:
function MyStats(t,f)
    sp=argmax(f); tp=t[sp]; fp=f[sp]
    s1,s2=minimum(findall(f.>fp/2)),maximum(findall(f.>fp/2))
    tfwh=t[s2]-t[s1]
    return tp
end;

 --- 

# (i) Panels 1A and 1B for single synaptic input   

In [7]:
# Time and space grid
dtA=0.001; TA=200; 
tA=collect(dtA:dtA:TA); ntA=length(tA); 
dxA=2; LA=2000; 
xA=collect(dxA:dxA:LA); nxA=length(xA);

# time and position of pulses
xAs=LA/2;  # midpoint
tAs=2*dtA; # just above zero

sAh=Int(xAs/dxA);

In [8]:
# Get a simulation pulse for the point and dendrite cases

UDsim,HeD,HiD=DendrShotSingleSynapseSim(xA,tA,xAs,tAs,game)
UPsim,HeP,HiP=PointShotSingleSynapseSim(tA,tAs,game);

UDsim=UDsim .-EL
UPsim=UPsim .-EL

# Three positions in addition to the zero one
xA1,xA2,xA3=50,100,200;
sA1,sA2,sA3=Int.(((xAs+xA1)/dxA,(xAs+xA2)/dxA,(xAs+xA3)/dxA));
UD0sim,UD1sim,UD2sim,UD3sim=UDsim[sAh,:],UDsim[sA1,:],UDsim[sA2,:],UDsim[sA3,:];

In [9]:
# Theory results

#(UDthe0,UDthe1,UDthe2,UDthe3)=DendShotSingleSynapseTheory.(t,(0.0,x1,x2,x3),game);
UD0=DendShotSingleSynapseTheory(tA,0.0,game) .-EL;
UD1=DendShotSingleSynapseTheory(tA,xA1,game) .-EL;
UD2=DendShotSingleSynapseTheory(tA,xA2,game) .-EL
UD3=DendShotSingleSynapseTheory(tA,xA3,game) .-EL;

UP=PointShotSingleSynapseTheory(tA,game).-EL;

In [10]:
tpD=[MyStats(tA,UD0),MyStats(tA,UD1),MyStats(tA,UD2),MyStats(tA,UD3)];
tpP=MyStats(tA,UP);

In [11]:
ttA=[0.1,2,5,10];
ssA=Int.(ttA/dtA)
xxA=xA.-1000;

In [12]:
#####################
# Summary
#####################

# UDsim is a space time simulation

# For panel 1A
# Use UDsim at timeslices given by ttA

# These are for panel 1B
# UD0sim, UD1sim etc are simulations for v versus time 
# UD0,UD1,UD2,UD3 are theory for v vesus time 


# UP is theory for point case


In [13]:
# find the maximum of the simulation

simmaxs=argmax(UDsim)[2]
themaxs=argmax(UD0)

simmaxt=round(dtA*simmaxs,digits=2)
themaxt=round(dtA*simmaxs,digits=2)

simmaxv=round(maximum(UDsim),digits=3)
themaxv=round(maximum(UD0),digits=3)

println("max t: $simmaxt $themaxt")
println("max v: $simmaxv $themaxv")


max t: 2.37 2.37
max v: 2.677 2.754


In [14]:
# Normalise the dendritic EPSPs

UDsim=UDsim/maximum(UDsim);
for UD in [UD1,UD2,UD3,UD0]
    UD[:]=UD[:]/maximum(UD0)
end

UP=UP/maximum(UP);

---

# (ii) Panels 1C and 1D for fluctuating input  

In [15]:
# Fluctuations are around the same steady state values used for the later figures.
# Need to find the mean voltage corresponding to target rate rstar.

rstar=rstarHz/K # the target rate in kHz

# The range of resting voltages
dEthe=0.1;
EEthe=collect(EL:dEthe:Vth)
nEEthe=length(EEthe)
aL=1/tauL; a0=aL/kappa
aae=((EEthe .-Ei)*a0 .- aL*(EL-Ei))/(Ee-Ei)
aai=((Ee .-EEthe)*a0 .- aL*(Ee-EL))/(Ee-Ei);

# Theoretical quantities
DV0EEthe,Dvv0EEthe,Ddvdv0EEthe,Dru0EEthe,Dvhe0EEthe,Dvhi0EEthe=(zeros(nEEthe) for j=1:6)
for k=1:nEEthe
    DV0EEthe[k],Dvhe0EEthe[k],Dvhi0EEthe[k],Dvv0EEthe[k],Ddvdv0EEthe[k],Dru0EEthe[k]=DendrSteadyTheory(aae[k],aai[k]);
end

# Linear interpolation for the mean voltage
GetxStar(x1,x2,y1,y2,ystar)=x1 + (x2-x1)*(ystar-y1)/(y2-y1)
sD2=minimum(findall(Dru0EEthe.>rstar)); sD1=sD2-1
DEstar=GetxStar(EEthe[sD1],EEthe[sD2],Dru0EEthe[sD1],Dru0EEthe[sD2],rstar)

# Get the various upcrossing parameters for these values
Daestar=((DEstar .-Ei)*a0 .- aL*(EL-Ei))/(Ee-Ei)
Daistar=((Ee .-DEstar)*a0 .- aL*(Ee-EL))/(Ee-Ei);
DV0Esthe,Dvhe0Esthe,Dvhi0Esthe,Dvv0Esthe,Ddvdv0Esthe,Dru0Esthe=DendrSteadyTheory(Daestar,Daistar);

print("Mean voltage for r*=$(rstarHz)Hz is E*=$(round(DEstar,digits=2))mV\n")
print("Actual rate at this mean voltage is $(round(K*Dru0Esthe,digits=3))Hz")


Mean voltage for r*=5.0Hz is E*=-57.12mV
Actual rate at this mean voltage is 4.999Hz

In [16]:
# Now run some simulations to show examples of the fluctuating voltage.

#dt=0.005; dx=10
dt=0.02; dx=20;
#dt=0.04; dx=25
#dt=0.2; dx=50;

T=1000; 
t=collect(dt:dt:T); nt=length(t); 
x=collect(dx:dx:L); nx=length(x);

# Gets data and rough figure. Set the seed:
Random.seed!(10);

Daestart=Daestar*ones(nt); Daistart=Daistar*ones(nt)
Dstatestart=(zeros(nx),zeros(nx),EL,0,0,zeros(nx))
bU,u,~,~,~,Dstatestart=DendrGaussApproxSim(x,t,Daestart,Daistart,Dstatestart);


In [17]:

#TP=670; TT=100; dTT=20; tvshift=15
#tt=collect(T-TT-TP:dTT:T-TP); ss=Int.(ceil.(tt/dt)); ntt=length(tt);

---   

# (iii) Figure 1 code

In [18]:
##################################################################
# Figure 1.
##################################################################

pygui(true)
fig=figure(figsize=1.6.*(3.375,3.125))

panelfs=14
labelfs=8
tickfs=8
lw=1.0
ms=3

##################################################################
# Positions
##################################################################
bx,tx=0.1,0.95 # top and bottom x edges
by,ty=0.1,0.925 # top and bottom y edges
gx,gy=0.12,0.12 # gap sizes

wx=(tx-bx-gx)/2
wy=(ty-by-gy)/2

xA,yA,wA,hA=bx,by+wy+gy,wx,wy
xB,yB,wB,hB=bx+wx+gx,by+wy+gy,wx,wy
xC,yC,wC,hC=bx,by,wx,wy
xD,yD,wD,hD=bx+wx+gx,by,wx,wy

sI=0.2
xBI,yBI,wBI,hBI=bx+wx+gx+sI,by+wy+gy+sI,wx-sI,wy-sI

##################################################################
# Panel 1A: Voltage by space for different times
##################################################################

ax1=plt.axes([xA,yA,wA,hA]);
fig.text(xA-0.08, 0.95, "A", fontsize=panelfs)
plot(xxA,UDsim[:,ssA[1]],"k-",linewidth=lw);
plot(xxA,UDsim[:,ssA[2]],"k-",linewidth=lw);
plot(xxA,UDsim[:,ssA[3]],"k-",linewidth=lw);
plot(xxA,UDsim[:,ssA[4]],"k-",linewidth=lw);
text(-35,0.42,L"0.5",fontsize=tickfs)
text(-35,0.36,L"\mathrm{ms}",fontsize=tickfs)
text(5,1.05,L"2\mathrm{ms}",fontsize=tickfs)
text(70,0.55,L"5\mathrm{ms}",fontsize=tickfs)
text(280,0.1,L"10\mathrm{ms}",fontsize=tickfs)

#fill_between(xxA,UDsim[:,ssA[1]],color="black",alpha=0.1)
#fill_between(xxA,UDsim[:,ssA[2]],color="darkgray",alpha=0.1)
#fill_between(xxA,UDsim[:,ssA[3]],color="gray",alpha=0.1)
#fill_between(xxA,UDsim[:,ssA[4]],color="lightgray",alpha=0.1)
yp=0.02
#plot(0,yp,"bo",50,yp,"go",100,yp,"yo",200,yp,"ro",markersize=ms)
axis([-400,400,0,1.2]); 
xticks(fontsize=tickfs);yticks(fontsize=tickfs)
xlabel(L"$\mathrm{Position}$ ($\mu$m)",fontsize=labelfs); 
ylabel(L"$\mathrm{Voltage}$ (mV)",fontsize=labelfs)

##################################################################
# Panel 1B: Voltage by space for different times
##################################################################

ax2=plt.axes([xB,yB,wB,hB]); 
fig.text(xB-0.08, 0.95, "B", fontsize=panelfs)
plot(tA,UP,"k:",linewidth=lw)
plot(tA,UD0,"k-",linewidth=lw)
plot(tA,UD1,"k-",linewidth=lw)
plot(tA,UD2,"k-",linewidth=lw)
plot(tA,UD3,"k-",linewidth=lw)
text(0,1.05,L"0\mu\mathrm{m}",fontsize=tickfs)
text(11,0.57,L"50\mu\mathrm{m}",fontsize=tickfs)
text(20,0.36,L"100\mu\mathrm{m}",fontsize=tickfs)
text(50,0.1,L"200\mu\mathrm{m}",fontsize=tickfs)

axis([-2,100,0,1.2])
ax2.spines["top"].set_visible(false)
ax2.spines["right"].set_visible(false)
xticks(fontsize=tickfs);yticks(fontsize=tickfs)
xlabel("Time (ms)",fontsize=labelfs); 
ylabel("Voltage (mV)",fontsize=labelfs)

##################################################################
# Panel 1B Inset: Voltage by space for different times
##################################################################

ax3=plt.axes([xBI,yBI,wBI,hBI]);
plot([0,xA1,xA2,xA3],tpD,"k-")
plot(0,tpD[1],"ko",markersize=ms)
plot(xA1,tpD[2],"ko",markersize=ms)
plot(xA2,tpD[3],"ko",markersize=ms)
plot(xA3,tpD[4],"ko",markersize=ms)
plot([0,xA3],tpP*[1,1],"k:",linewidth=lw)
xticks(fontsize=tickfs);yticks(fontsize=tickfs)
xlabel(L"Distance ($\mu$m)",fontsize=labelfs); 
ylabel("Peak time (ms)",fontsize=labelfs)
axis([-2,xA3*1.1,0,tpD[end]*1.1]);

##################################################################
# Panel 1C: Voltage by space for different times
##################################################################
TP=670; TT=100; dTT=20; tvshift=15
tt=collect(T-TT-TP:dTT:T-TP); ss=Int.(ceil.(tt/dt)); ntt=length(tt);

s0=findall((x.-1000).==0) # find the zero point

gbit=hC/10
hCbit=(hC-2*gbit)/3

ax41=plt.axes([xC,yC,wC,hCbit]);
plot(x.-1000,u[ss[5],:].+bU[ss[5]],"k-",linewidth=lw)
plot(x[s0].-1000,u[ss[5],s0].+bU[ss[5]],"ko",markersize=ms)
xlabel(L"Position ($\mu$m)",fontsize=labelfs); 
xticks(fontsize=tickfs);yticks(fontsize=tickfs)
text(0,-63,L"3",fontsize=tickfs)

ax42=plt.axes([xC,yC+hCbit+gbit,wC,hCbit])
plot(x.-1000,u[ss[4],:].+bU[ss[4]],"k-",linewidth=lw)
plot(x[s0].-1000,u[ss[4],s0].+bU[ss[4]],"ko",markersize=ms)
ylabel("Voltage (mV)",fontsize=labelfs)
xticks(fontsize=tickfs);yticks(fontsize=tickfs)
text(0,-55,L"2",fontsize=tickfs)

ax43=plt.axes([xC,yC+2hCbit+2gbit,wC,hCbit])
plot(x.-1000,u[ss[3],:].+bU[ss[3]],"k-",linewidth=lw)
plot(x[s0].-1000,u[ss[3],s0].+bU[ss[3]],"ko",markersize=ms)
xticks(fontsize=tickfs);yticks(fontsize=tickfs)
text(0,-56,L"1",fontsize=tickfs)

for ax in [ax41,ax42,ax43]
    #ax.plot(x.-1000,EL*ones(size(x)),"k:",linewidth=lw)
    ax.plot(x.-1000,Vth*ones(size(x)),"k:",linewidth=lw)
    ax.axis([-400,400,EL-5,Vth+5])
end

ax42.set_xticklabels([],fontsize=tickfs)
ax43.set_xticklabels([],fontsize=tickfs)

fig.text(xC-0.08, 0.5, "C", fontsize=panelfs)


##################################################################
# Panel 1D: Voltage by space for different times
##################################################################

ax5=plt.axes([xD,yD,wD,hD]);
fig.text(xD-0.08, 0.5, "D", fontsize=panelfs)
sh=Int(L/(2dx));
UU=bU.+u[:,sh]
plot(t,UU,"-k",linewidth=lw)
plot([0,1000],Vth*[1,1],"k:",linewidth=lw)
text(310,-50+0.5,L"V_\mathrm{th}",fontsize=tickfs)
plot(tt,UU[ss],"ok",markersize=ms)
text(270,-58,L"1",fontsize=tickfs)
text(290,-47,L"2",fontsize=tickfs)
text(310,-59,L"3",fontsize=tickfs)
axis([260,320,EL-5,Vth+5]);
ax2.spines["top"].set_visible(false)
ax2.spines["right"].set_visible(false)

xticks(fontsize=tickfs);yticks(fontsize=tickfs)
xlabel("Time (ms)",fontsize=labelfs); 
ylabel("Voltage (mV)",fontsize=labelfs);

matplotlib.pyplot.arrow(280, -48, 4.5, -1.5,width=0.1,color="k")

for ax in [ax1,ax2,ax3,ax5]
    ax.spines["top"].set_visible(false)
    ax.spines["right"].set_visible(false)
end

savefig("Fig1.pdf")

In [19]:
close("all")